<a href="https://colab.research.google.com/github/punkmic/Unsupervised-Sentiment-Analysis---Comparisen-analysis/blob/master/Unsupervised_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intro**

## **Install Dependecies**

In [1]:
# install dependecies here
!pip install langdetect  # for language detection
!pip install googletrans # for text translate
!pip install diagrams # for visualize the workflow
!pip install graphviz # for visualize the workflow
!pip install beautifulsoup4 # for web scraping
!pip install selenium # for automate web browser interaction

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some code need to use selenium in Colab

In [ ]:
# https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## **Load Depencies**

In [5]:
# load dependecies here
from langdetect import detect as dt
from diagrams import Diagram as dg
from googletrans import Translator as tls
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## **Web Scraping** 

In [22]:
# web scraping here
#class Page:
#  def __init__(self, driver):
#   self.driver = driver
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get("https://www.reclameaqui.com.br/")
html = wd.page_source
page = bs(html,'html.parser')

In [23]:
page.text

'\nAccess Denied\n\nAccess Denied\n \nYou don\'t have permission to access "http://www.reclameaqui.com.br/" on this server.\nReference #18.1da7cb17.1663089502.7996fe7\n\n\n'

## **Save Data to Mysql for Offline Use**

In [ ]:
# save to mysql

## **Load Dataset**

In [ ]:
# load dataset here

## **Text Pre-Processing**

* Lower Case conversion
* Removing Punctuations
* Stop Words Removal
* Rare Words Removal
* Spelling correction
* Tokenization
* Lemmatization



In [ ]:
# pre-processing text here

## **Feature Extraction**

In [ ]:
# Bag of words, TF-IDF

## **Modeling**

In [ ]:
# create and train models

## **Save Models to Google Cloud Storage**

In [ ]:
# import google cloud dependencies
from google.colab import auth
import uuid # for generate a unique identification for google bucket
# Define a project id in google cloud
project_id = '<project_ID>'

auth.authenticate_user()
# configure gsutil
## !gcloud config set project {project_id}
# set bucket name
##backet_name = f'sample-bucket-{uuid.uuid1()}'
## !gsuit mb gs://{bucket_name}

In [ ]:
# upload model to Google Cloud Storage
!gsuit cp /tmp/name_of_file.txt gs://{bucket_name}/

# location of model
download_location = f"https://console.cloud.google.com/storage/browser?project={project_id}"

# donwload model from Google Cloud Storage
!gsuit cp gs://{bucket_name}/{filename} {download_location}

## **References**


[LangDetect](https://pypi.org/project/langdetect/) <br/>
[Diagrams](https://pypi.org/project/diagrams/) <br/>
[Graphviz](https://pypi.org/project/graphviz/) <br/>
[GoogleTrans](https://pypi.org/project/googletrans/) <br/>
[Beautifulsoap4](https://pypi.org/project/beautifulsoup4/) <br/>
[OpLexicon](https://www.inf.pucrs.br/linatural/wordpress/recursos-e-ferramentas/oplexicon/)